In [22]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.layers import *
from keras.models import Sequential
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from tensorflow.keras.callbacks import EarlyStopping
import pickle as pkl



In [23]:
df = pd.read_excel('Online Retail.xlsx')
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [25]:
df.isna().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [26]:
df['Description'].ffill(inplace=True)
df.dropna(subset=['CustomerID'], inplace=True)
df.isna().sum()

C:\Users\ShehabYousef\AppData\Local\Temp\ipykernel_14420\888066231.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Description'].ffill(inplace=True)


InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [27]:
df['Quantity']= df['Quantity'].abs()

In [28]:
df['total_price'] = df['Quantity'] * df['UnitPrice']

In [29]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [30]:
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)

# Add RFM columns to original df
df['Recency'] = df.groupby('CustomerID')['InvoiceDate'].transform(lambda x: (snapshot_date - x.max()).days)
df['Frequency'] = df.groupby('CustomerID')['InvoiceNo'].transform('nunique')
df['Monetary'] = df.groupby('CustomerID')['total_price'].transform('sum')

df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_price,Recency,Frequency,Monetary
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,302,35,5493.79
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,302,35,5493.79
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,302,35,5493.79
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,302,35,5493.79
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,302,35,5493.79
...,...,...,...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20,1,4,862.81
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60,1,4,862.81
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,1,4,862.81
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60,1,4,862.81


In [31]:
df['AverageOrderValue'] = df['Monetary'] / df['Frequency']
df['AvgQuantity'] = df.groupby('CustomerID')['Quantity'].transform('mean')
df.drop_duplicates(subset=['CustomerID'], inplace=True)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,total_price,Recency,Frequency,Monetary,AverageOrderValue,AvgQuantity
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,302,35,5493.79,156.965429,5.682692
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54.08,32,18,3395.98,188.665556,7.280612
26,536370,22728,ALARM CLOCK BAKELIKE PINK,24,2010-12-01 08:45:00,3.75,12583.0,France,90.00,3,18,7375.42,409.745556,20.362550
46,536371,22086,PAPER CHAIN KIT 50'S CHRISTMAS,80,2010-12-01 09:00:00,2.55,13748.0,United Kingdom,204.00,96,5,948.25,189.650000,15.678571
65,536374,21258,VICTORIAN SEWING BOX LARGE,32,2010-12-01 09:09:00,10.95,15100.0,United Kingdom,350.40,330,6,1116.90,186.150000,17.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536969,581233,22178,VICTORIAN GLASS HANGING T-LIGHT,6,2011-12-08 10:33:00,1.95,13436.0,United Kingdom,11.70,2,1,196.89,196.890000,6.333333
537255,581241,22922,FRIDGE MAGNETS US DINER ASSORTED,12,2011-12-08 10:58:00,0.85,15520.0,United Kingdom,10.20,2,1,343.50,343.500000,17.444444
538064,581385,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,24,2011-12-08 13:11:00,3.75,13298.0,United Kingdom,90.00,1,1,360.00,360.000000,48.000000
538812,581416,22466,FAIRY TALE COTTAGE NIGHT LIGHT,1,2011-12-08 14:58:00,1.95,14569.0,United Kingdom,1.95,1,1,227.39,227.390000,6.583333


In [32]:
df.drop(columns=['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'total_price'], inplace=True)
df

,CustomerID,Country,Recency,Frequency,Monetary,AverageOrderValue,AvgQuantity
0,17850.0,United Kingdom,302,35,5493.79,156.965429,5.682692
9,13047.0,United Kingdom,32,18,3395.98,188.665556,7.280612
26,12583.0,France,3,18,7375.42,409.745556,20.362550
46,13748.0,United Kingdom,96,5,948.25,189.650000,15.678571
65,15100.0,United Kingdom,330,6,1116.90,186.150000,17.000000
...,...,...,...,...,...,...,...
536969,13436.0,United Kingdom,2,1,196.89,196.890000,6.333333
537255,15520.0,United Kingdom,2,1,343.50,343.500000,17.444444
538064,13298.0,United Kingdom,1,1,360.00,360.000000,48.000000
538812,14569.0,United Kingdom,1,1,227.39,227.390000,6.583333


In [33]:
df[df['Frequency'] == 1].shape[0]

1313

## Building the model


In [34]:
x=df[['Recency', 'Monetary','AvgQuantity']]
y = (df['Frequency'] > 1).astype(int)


X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
smote = SMOTE()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)


In [35]:
print(y_train_res.value_counts())


Frequency
1    2436
0    2436
Name: count, dtype: int64


In [36]:
model = Sequential()

model.add(Dense(128, activation='relu', input_shape=(X_train_res.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))


c:\Users\ShehabYousef\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [37]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [38]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)



In [39]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_5 (Dense)                 │ (None, 128)            │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,393 (44.50 KB)

 Trainable params: 11,393 (44.50 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
history = model.fit(
    X_train_res, y_train_res,
    epochs=20, 
    batch_size=64,
    validation_split=0.2,
)

Epoch 1/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6736 - loss: 0.6297 - val_accuracy: 0.4944 - val_loss: 0.8186
Epoch 2/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7072 - loss: 0.5604 - val_accuracy: 0.5067 - val_loss: 0.7211
Epoch 3/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7182 - loss: 0.5168 - val_accuracy: 0.6841 - val_loss: 0.6715
Epoch 4/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7567 - loss: 0.4767 - val_accuracy: 0.7846 - val_loss: 0.6057
Epoch 5/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7716 - loss: 0.4567 - val_accuracy: 0.8359 - val_loss: 0.5875
Epoch 6/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7911 - loss: 0.4456 - val_accuracy: 0.8092 - val_loss: 0.5621
Epoch 7/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7963 - loss: 0.4358 - val_accuracy: 0.8123 - val_loss: 0.5853
Epoch 8/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8034 - loss: 0.4266 - val_accuracy: 0.7815 - val_loss:

In [41]:
y_pred_probs = model.predict(X_test)
y_pred = (y_pred_probs > 0.5).astype(int)

from sklearn.metrics import accuracy_score, classification_report

acc = accuracy_score(y_test, y_pred)
print("Test Accuracy:", acc)
print(classification_report(y_test, y_pred))



28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Test Accuracy: 0.832
              precision    recall  f1-score   support

           0       0.70      0.73      0.71       252
           1       0.89      0.87      0.88       623

    accuracy                           0.83       875
   macro avg       0.79      0.80      0.80       875
weighted avg       0.83      0.83      0.83       875



In [42]:
model.save("model.h5")  